# Explore a Genus

Explore prophages in a genus. Initially Mycobacteria tuberculosis, but then we might do others like Acinetobacter

In [1]:
# A lot of this is not used, but we import it so we have it later!
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns
import numpy as np

import functools

import math
import re

sys.path.insert(0,'../')
from PhiSpyAnalysis import theils_u, DateConverter, printmd
from PhiSpyAnalysis import read_phages, read_gtdb, read_checkv, read_base_pp, read_categories, read_metadata, read_gbk_metadata, read_phage_locations
from PhiSpyAnalysis import bigtable, biglocations

from scipy.stats import pearsonr, f_oneway
from sklearn.linear_model import LinearRegression
from sklearn import decomposition
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, tukeyhsd, MultiComparison
from statsmodels.multivariate.manova import MANOVA


import subprocess
import gzip


In [2]:
bl = biglocations()

In [3]:
# GENUS="Mycobacterium tuberculosis"
GENUS="Acinetobacter baumannii"

In [4]:
mtb = bl[bl['genome_name'].str.contains(GENUS)]

In [5]:
mtb

,assembly_accession,GENOMEID,Contig,Start,Stop,Length,# CDS,Decision,genome_id,genome_name,...,trna_aa_count,trna_count,trna_selenocysteine_count,domain,phylum,class,order,family,genus,species
1203555,GCA_000315155.1,GCA_000315155.1,AMST01000070.1,349,29226,28878,37,Kept,903916.30000,Acinetobacter baumannii WC-136,...,20,67,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1203556,GCA_000315155.1,GCA_000315155.1,AMST01000130.1,16404,41783,25380,31,Kept,903916.30000,Acinetobacter baumannii WC-136,...,20,67,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1203557,GCA_000315155.1,GCA_000315155.1,AMST01000079.1,5939,22861,16923,9,Kept,903916.30000,Acinetobacter baumannii WC-136,...,20,67,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1203558,GCA_000315155.1,GCA_000315155.1,AMST01000045.1,7348,17397,10050,8,Dropped. No genes were identified as phage genes,903916.30000,Acinetobacter baumannii WC-136,...,20,67,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1203559,GCA_000315155.1,GCA_000315155.1,AMST01000118.1,120,4341,4222,7,Dropped. No genes were identified as phage genes,903916.30000,Acinetobacter baumannii WC-136,...,20,67,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134779,GCA_902713735.1,GCA_902713735.1,CACTJA010000139.1,74700,76043,1344,1,Dropped. Region too small (Not enough genes),470.11108,Acinetobacter baumannii strain AC3-M,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
3134780,GCA_902713735.1,GCA_902713735.1,CACTJA010000121.1,36242,37201,960,1,Dropped. Region too small (Not enough genes),470.11108,Acinetobacter baumannii strain AC3-M,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
3134781,GCA_902713735.1,GCA_902713735.1,CACTJA010000107.1,21250,23433,2184,1,Dropped. Region too small (Not enough genes),470.11108,Acinetobacter baumannii strain AC3-M,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
3134782,GCA_902713735.1,GCA_902713735.1,CACTJA010000068.1,32367,33287,921,1,Dropped. Region too small (Not enough genes),470.11108,Acinetobacter baumannii strain AC3-M,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii


In [6]:
mtb[(mtb['Length'] > 30000) & (mtb['Decision'] == 'Kept')]

,assembly_accession,GENOMEID,Contig,Start,Stop,Length,# CDS,Decision,genome_id,genome_name,...,trna_aa_count,trna_count,trna_selenocysteine_count,domain,phylum,class,order,family,genus,species
1203627,GCA_000681535.1,GCA_000681535.1,JMNQ01000014.1,18748,56438,37691,8,Kept,1.310642e+06,Acinetobacter baumannii 1451147,...,20,130,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1206435,GCA_900495015.1,GCA_900495015.1,UFIV01000003.1,22107,69222,47116,57,Kept,4.705505e+02,Acinetobacter baumannii strain 4300STDY7045719,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1206436,GCA_900495015.1,GCA_900495015.1,UFIV01000004.1,70884,106251,35368,54,Kept,4.705505e+02,Acinetobacter baumannii strain 4300STDY7045719,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1206437,GCA_900495015.1,GCA_900495015.1,UFIV01000003.1,177567,214593,37027,37,Kept,4.705505e+02,Acinetobacter baumannii strain 4300STDY7045719,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
1206438,GCA_900495015.1,GCA_900495015.1,UFIV01000030.1,2259,32829,30571,9,Kept,4.705505e+02,Acinetobacter baumannii strain 4300STDY7045719,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter pittii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134709,GCA_902705405.1,GCA_902705405.1,CACSGY010000012.1,448285,492315,44031,56,Kept,4.701109e+02,Acinetobacter baumannii strain 571B5-12EESBL,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
3134710,GCA_902705405.1,GCA_902705405.1,CACSGY010000006.1,51883,100556,48674,52,Kept,4.701109e+02,Acinetobacter baumannii strain 571B5-12EESBL,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
3134711,GCA_902705405.1,GCA_902705405.1,CACSGY010000008.1,235256,271129,35874,36,Kept,4.701109e+02,Acinetobacter baumannii strain 571B5-12EESBL,...,20,63,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
3134726,GCA_902705485.1,GCA_902705485.1,CACSGU010000007.1,1288961,1328849,39889,53,Kept,4.701109e+02,Acinetobacter baumannii strain KPC-SM-125,...,20,64,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii


In [7]:
pd.unique(mtb['Decision'])

array(['Kept', 'Dropped. No genes were identified as phage genes',
       'Dropped. Region too small (Not enough genes)'], dtype=object)

In [8]:
keptlq = mtb[mtb['Decision'] == 'Dropped. No genes were identified as phage genes'][['GENOMEID', 'Contig', 'Start', 'Stop', 'Length', '# CDS', 'Decision',  'genome_name', 'species', 'taxon_id', 'strain', 'mlst', 'bioproject_accession', 'biosample_accession', ]]
with open("../outputs/Acinetobacter_LQpp.tsv", 'w') as out:
    keptlq.to_csv(out, sep="\t")
keptlq

,GENOMEID,Contig,Start,Stop,Length,# CDS,Decision,genome_name,species,taxon_id,strain,mlst,bioproject_accession,biosample_accession
1203558,GCA_000315155.1,AMST01000045.1,7348,17397,10050,8,Dropped. No genes were identified as phage genes,Acinetobacter baumannii WC-136,Acinetobacter pittii,48296,WC-136,MLST.Acinetobacter_baumannii_2.64,PRJNA53431,SAMN00761240
1203559,GCA_000315155.1,AMST01000118.1,120,4341,4222,7,Dropped. No genes were identified as phage genes,Acinetobacter baumannii WC-136,Acinetobacter pittii,48296,WC-136,MLST.Acinetobacter_baumannii_2.64,PRJNA53431,SAMN00761240
1203629,GCA_000681535.1,JMNQ01000001.1,8591,15590,7000,6,Dropped. No genes were identified as phage genes,Acinetobacter baumannii 1451147,Acinetobacter pittii,1310642,1451147,MLST.Acinetobacter_baumannii_1.1067,PRJNA194718,SAMN02692951
1487050,GCA_000694975.1,KK737786.1,1323959,1412972,89014,105,Dropped. No genes were identified as phage genes,Acinetobacter baumannii BIDMC 57,Acinetobacter nosocomialis,1439317,BIDMC 57,MLST.Acinetobacter_baumannii_2.530,PRJNA234145,SAMN02581278
1487051,GCA_000694975.1,KK737786.1,2244197,2281125,36929,35,Dropped. No genes were identified as phage genes,Acinetobacter baumannii BIDMC 57,Acinetobacter nosocomialis,1439317,BIDMC 57,MLST.Acinetobacter_baumannii_2.530,PRJNA234145,SAMN02581278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134743,GCA_902705505.1,CACSGV010000005.1,318,25033,24716,23,Dropped. No genes were identified as phage genes,Acinetobacter baumannii strain KPC-SM-17a,Acinetobacter baumannii,470,KPC-SM-17a,NaN,PRJEB35515,SAMEA6242923
3134745,GCA_902705505.1,CACSGV010000028.1,74199,88010,13812,18,Dropped. No genes were identified as phage genes,Acinetobacter baumannii strain KPC-SM-17a,Acinetobacter baumannii,470,KPC-SM-17a,NaN,PRJEB35515,SAMEA6242923
3134773,GCA_902713735.1,CACTJA010000046.1,39862,63023,23162,45,Dropped. No genes were identified as phage genes,Acinetobacter baumannii strain AC3-M,Acinetobacter baumannii,470,AC3-M,"MLST.Acinetobacter_baumannii_1.231,MLST.Acinet...",PRJEB35978,SAMEA6436546
3134774,GCA_902713735.1,CACTJA010000160.1,4748,18823,14076,24,Dropped. No genes were identified as phage genes,Acinetobacter baumannii strain AC3-M,Acinetobacter baumannii,470,AC3-M,"MLST.Acinetobacter_baumannii_1.231,MLST.Acinet...",PRJEB35978,SAMEA6436546


In [9]:
dict(mtb.loc[1203626])

{'assembly_accession': 'GCA_000681535.1',
 'GENOMEID': 'GCA_000681535.1',
 'Contig': 'JMNQ01000018.1',
 'Start': 18126,
 'Stop': 38764,
 'Length': 20639,
 '# CDS': 28,
 'Decision': 'Kept',
 'genome_id': 1310642.3,
 'genome_name': 'Acinetobacter baumannii 1451147',
 'organism_name': nan,
 'taxon_id': 1310642,
 'genome_status': 'WGS',
 'strain': '1451147',
 'serovar': nan,
 'biovar': nan,
 'pathovar': nan,
 'mlst': 'MLST.Acinetobacter_baumannii_1.1067',
 'other_typing': nan,
 'culture_collection': nan,
 'type_strain': nan,
 'completion_date': '2014-05-06T00:00:00Z',
 'publication': '27458211',
 'bioproject_accession': 'PRJNA194718',
 'biosample_accession': 'SAMN02692951',
 'genbank_accessions': 'JMNQ00000000',
 'refseq_accessions': '-',
 'sequencing_centers': 'Institute for Genome Sciences',
 'sequencing_status': nan,
 'sequencing_platform': 'Illumina',
 'sequencing_depth': '58.66x',
 'assembly_method': 'MaSuRCA v. 2.0.3.1',
 'chromosomes': nan,
 'plasmids': nan,
 'contigs': 539.0,
 'seq